In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=4


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
import pandas as pd
from ast import literal_eval

In [14]:
df = pd.read_csv('test_subset.csv')
df

,user,instance_id,text,sentiment,negative object_spans,negative spans,positive spans,positive object_spans
0,ekaterini@gmail.com,898,Nur Politiker wollen Krieg ! Was wenn keiner m...,negative,"['Politiker ', 'Krieg ']","['Was wenn keiner mehr Bock hat', 'zu sterben'...",...,...
1,ekaterini@gmail.com,833,Der Klimawandel ist ein völlig normaler Prozes...,positive,...,...,völlig normaler Prozess,Klimawandel
2,ekaterini@gmail.com,1428,Man will die Stadthalle für einen #Bürgerdialo...,negative,Demokraten,"['missliebige Partei ', 'Was denken die']",...,...
3,ekaterini@gmail.com,431,"Besonders beliebt ist auch ""Wir mussten währen...",negative,Corona,die während Corona auch gern auf die Maske ver...,...,...
4,ekaterini@gmail.com,1315,Die USA sollten sich mäßigen Israel zu ermahne...,positive,...,...,handelt moralisch richtig,"['Israel ', 'Israel ']"
...,...,...,...,...,...,...,...,...
295,ekaterini@gmail.com,1134,Auch Israel begeht Kriegsverbrechen an der Ziv...,negative,Israel,"['begeht Kriegsverbrechen', 'aber Kritik an Is...",...,...
296,ekaterini@gmail.com,127,Sogenannte Zivilisten? geht's noch?,negative,...,geht's noch?,...,...
297,ekaterini@gmail.com,366,"Ganz normal, dass wir in Zeiten der Klimakrise...",negative,Klimakrise,"['hypokapitalistischen', 'Nice one, Humanity',...",...,...
298,ekaterini@gmail.com,473,Kleine Nachhilfe: Ein paar hunderttausend Grün...,negative,Grüne,eine Klatsche bekommen habt,...,...


In [5]:
#creating texts for which to evaluate sentiment
texts = list(df.text)

In [ ]:
#Using the fine-tuned model to make predictions
load_in_4bit = True
dtype = None
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Hina541/fine-tuned-mistralModel_tokenClassification_10epochs", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 512,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    
    )
    FastLanguageModel.for_inference(model)
#zero-shot after fine-tuning
prompt = f"Classify the sentiment of each word in the texts into ONE of the two classes: negative or positive. Split the answer in a dictionary with two keys:Tokens and Labels. Text:"

answers = []
#text_list =[ "Ukraine deckt auf 😂😂😂😂😂\nDie mussten was sagen, nicht das man denkt der Staat wäre korrupt und es würde keinen Kampf dagegen geben ☝️\n\nKorrupt bleibt korrupt - auch die, da wird sich auch nichts änder 😆",'Nur Politiker wollen Krieg ! Was wenn keiner mehr Bock hat für diese 5 Politiker zu sterben warum drehen wir den Spieß nich einfach um ? 😕 frage für n Freund']
for text in texts:
    chat_template = [{"role" : "user", "content" : f"{prompt} {text}"}]

    chat_prompt = tokenizer.apply_chat_template(chat_template, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(chat_prompt, add_special_tokens=False, return_tensors="pt", max_length=512,padding= True)
    outputs = model.generate(input_ids=inputs.to("cuda"), max_new_tokens = 512)
    answer = tokenizer.decode(outputs[0])
    answers.append(answer)
    #print(answer)
print(len(answers))

In [ ]:
#Processing the outputs of above results, to generate a list of rows with 'tokens and labels'
rows = []
not_proper = []
for answer in answers:
    #rsplit instead split
    if len(answer.split('[/INST]')[1].split(':'))>2:
        tokens=[]
        labels = []
        for item in answer.split('[/INST]')[1].split(':')[1].split(','):
            if item.strip() != "'Labels'" and item.strip() != "'" :
                tokens.append(item.strip(" '' ").strip('[').strip(']').strip("'"))
        for item in answer.split('[/INST]')[1].split(':')[2].split('}')[0].split(','):
            labels.append(item.strip(" '' ").strip('[').strip(']').strip("'"))
        rows.append({'tokens':tokens,'labels':labels})
    else:
        rows.append('NA')
        not_proper.append(answer)
print(len(not_proper))
print(len(rows))

In [15]:
#Adding the above generated tokens and labels to our dataframe and saving it for future reference
df['token_and_labels_FromMistralLLM'] = rows
df

,user,instance_id,text,sentiment,negative object_spans,negative spans,positive spans,positive object_spans,token_and_labels_FromMistralLLM
0,ekaterini@gmail.com,898,Nur Politiker wollen Krieg ! Was wenn keiner m...,negative,"['Politiker ', 'Krieg ']","['Was wenn keiner mehr Bock hat', 'zu sterben'...",...,...,"{'tokens': ['Politiker', 'Krieg', 'nicht', 'ei..."
1,ekaterini@gmail.com,833,Der Klimawandel ist ein völlig normaler Prozes...,positive,...,...,völlig normaler Prozess,Klimawandel,"{'tokens': ['Klimawandel', 'ein', 'völlig', 'n..."
2,ekaterini@gmail.com,1428,Man will die Stadthalle für einen #Bürgerdialo...,negative,Demokraten,"['missliebige Partei ', 'Was denken die']",...,...,"{'tokens': ['die', 'Stadthalle', 'für', 'einen..."
3,ekaterini@gmail.com,431,"Besonders beliebt ist auch ""Wir mussten währen...",negative,Corona,die während Corona auch gern auf die Maske ver...,...,...,"{'tokens': ['Corona', 'auf', 'so', 'viel', 've..."
4,ekaterini@gmail.com,1315,Die USA sollten sich mäßigen Israel zu ermahne...,positive,...,...,handelt moralisch richtig,"['Israel ', 'Israel ']","{'tokens': ['USA', 'sollten', 'sich', 'mäßigen..."
...,...,...,...,...,...,...,...,...,...
295,ekaterini@gmail.com,1134,Auch Israel begeht Kriegsverbrechen an der Ziv...,negative,Israel,"['begeht Kriegsverbrechen', 'aber Kritik an Is...",...,...,"{'tokens': ['Israel', 'begeht', 'Kriegsverbrec..."
296,ekaterini@gmail.com,127,Sogenannte Zivilisten? geht's noch?,negative,...,geht's noch?,...,...,"{'tokens': ['Sogenannte', 'Zivilisten'], 'labe..."
297,ekaterini@gmail.com,366,"Ganz normal, dass wir in Zeiten der Klimakrise...",negative,Klimakrise,"['hypokapitalistischen', 'Nice one, Humanity',...",...,...,"{'tokens': ['Klimakrise', 'hypokapitalistische..."
298,ekaterini@gmail.com,473,Kleine Nachhilfe: Ein paar hunderttausend Grün...,negative,Grüne,eine Klatsche bekommen habt,...,...,"{'tokens': ['hunderttausend', 'Grüne', 'nicht'..."


In [ ]:
#dropping the rows where the length of tokens and labels are not same (There were some wrong outputs generated by mistral, gemma did not have such outputs)
for i in range(len(df)):
    tokens = df['token_and_labels_FromMistralLLM'][i]['tokens']
    labels = df['token_and_labels_FromMistralLLM'][i]['labels']
    if len(tokens) != len(labels):
        df.drop(i, inplace=True)
        print(len(tokens))
        print(len(labels))


In [29]:
df.to_csv('Results_From_Mistral_withTokensAndLabelsAfterFineTuningTokenClassification-new.csv', index = False)

In [18]:
#You can either read the csv or just use our dataframe df
df = pd.read_csv('Results_From_Mistral_withTokensAndLabelsAfterFineTuningTokenClassification-new.csv', converters = {'token_and_labels_FromMistralLLM':literal_eval} )

In [30]:
#pre-processing the data so that the spans are not strings but lists
df.rename(columns={'negative object_spans':'negative_object_spans','negative spans':'negative_spans','positive object_spans':'positive_object_spans','positive spans':'positive_spans'}, inplace=True)
new_list_positive_object_spans = []
new_list_positive_spans = []
new_list_negative_object_spans = []
new_list_negative_spans =[]

list_positive_object_spans = df.positive_object_spans
list_positive_spans = list(df.positive_spans)
list_negative_object_spans = df.negative_object_spans
list_negative_spans = list(df.negative_spans)

#print(list_negative_object_spans)
#print(list_negative_spans)
#print(list_positive_object_spans)

for item in list_positive_object_spans:
    if not '[' in item:
        new_list_positive_object_spans.append(item.split())
    else:
        new_list_positive_object_spans.append(item)

for item in list_positive_spans:
    if not '[' in item:
        new_list_positive_spans.append(item.split())
    else:
        new_list_positive_spans.append(item)

for item in list_negative_object_spans:
    if not '[' in item:
        new_list_negative_object_spans.append(item.split())
    else:
        new_list_negative_object_spans.append(item)

for item in list_negative_spans:
    if not '[' in item:
        new_list_negative_spans.append(item.split())
    else:
        new_list_negative_spans.append(item)

#print(new_list_positive_object_spans)

In [31]:
df['negative_object_spans'] = new_list_negative_object_spans
df['negative_spans'] = new_list_negative_spans
df['positive_object_spans'] = new_list_positive_object_spans
df['positive_spans'] = new_list_positive_spans

### Creating data for NER evaluation , where I check if the tokens are in spans , if yes then add the actual and predicted sentiment, if not then add 'O' as in outer

In [ ]:

predicted_labels = []
actual_labels = []
words_in_spans = []
for i in range(len(df)):
    tokens = df['token_and_labels_FromMistralLLM'][i]['tokens']
    pred_labels_df = df['token_and_labels_FromMistralLLM'][i]['labels']
    for index,token in enumerate(tokens):
        if token.strip() in df['negative_object_spans'][i] or token.strip() in df['negative_spans'][i] or token.strip() in df['positive_object_spans'][i] or token.strip() in df['positive_spans'][i]:
            #print(token)
            actual_labels.append(df['sentiment'][i])
            words_in_spans.append(token)
            #for label in pred_labels_df:
            predicted_labels.append(pred_labels_df[index])
        else:
            words_in_spans.append(token)
            actual_labels.append('O')
            predicted_labels.append('O')
    words_in_spans.append(' ')
    predicted_labels.append(' ')
    actual_labels.append(' ')
print(len(predicted_labels))
print(len(actual_labels))
print(len(words_in_spans))

In [ ]:
df_ner_LLM = pd.DataFrame(columns=['tokens','actual','predicted'])
df_ner_LLM['tokens'] = words_in_spans
df_ner_LLM['actual'] = actual_labels
df_ner_LLM['predicted'] = predicted_labels
df_ner_LLM.head(50)

In [36]:
#Making sure that there are no <eos> attached to the predicted sentiments
count=0
for i in range(len(df_ner_LLM)):
    df_ner_LLM.loc[i,'predicted'] = str(df_ner_LLM['predicted'][i]).strip(' ')
    if '<eos>' in df_ner_LLM['predicted'][i]:
        df_ner_LLM.loc[i,'predicted'] = str(df_ner_LLM['predicted'][i])[:-5]
        count = count + 1
print(count)


0


In [37]:
#Making sure to refer neutral as 'O' as in outer if present
for i in range(len(df_ner_LLM)):
    if df_ner_LLM['predicted'][i] == 'neutral':
        df_ner_LLM.loc[i,'predicted'] = 'O'

In [38]:
#saving the results as csv file to perform NER evaluation using conlleval
df_ner_LLM.to_csv('ResultsOFNEREvaluationLLMMistralAfterFineTuningWithTokenClassification-Latest.csv',header=False,index=False, sep='\t')

### Below are the results of NER Evaluation
#### python conlleval_perl.py -r < ResultsOFNEREvaluationLLMMistralAfterFineTuningWithTokenClassification-Latest.csv  -- command used for NER evaluation
#### processed 3679 tokens with 1738 phrases; found: 1738 phrases; correct: 1533.
#### accuracy:  94.43%; precision:  88.20%; recall:  88.20%; FB1:  88.20
         #### negative: precision:  93.25%; recall:  92.93%; FB1:  93.09  1481
         #### positive: precision:  59.14%; recall:  60.32%; FB1:  59.72  257